In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Seg3D/KiTS2019/kits19'

/content/drive/MyDrive/Seg3D/KiTS2019/kits19


In [ ]:
!pip install torchio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.8/172.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: rich
    Found existing installation: rich 13.3.4
    Uninstalling rich-13.3.4:
      Successfully uninstalled rich-13.3.4


In [ ]:
from dataclasses import dataclass, field
from typing import Optional, Union

import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from segment.utils.hf_argparser import HfArgumentParser
from segment.utils.file_utils import logger, load_json_file, read_yaml_file
from segment.data.data_readers.data_reader import DatasetReader
from segment.data.augs import aug_maps


In [ ]:
data = load_json_file("/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/train_val_data.json")["data"]
df = pd.DataFrame(data)
df.head(5)

,case_id,new_vol_path,new_seg_path,fold
0,case_00002,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,5.0
1,case_00003,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,9.0
2,case_00001,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,4.0
3,case_00000,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,6.0
4,case_00004,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/o...,5.0


In [ ]:
config = read_yaml_file("configs/preprocess_pipeline.yaml")["create_dataset"]


In [ ]:
from segment.data.data_readers.data_reader import DatasetReader
            
def check_nonzero_slice(item):
    nonero_slice = DatasetReader.locate_nonzero_slice(item, return_rate=False)
    return nonero_slice

def get_item(idx):
    row = df.iloc[idx]
    vol_path = row["new_vol_path"]
    seg_path = row["new_seg_path"]
    vol_arr = np.load(vol_path)
    seg_arr = np.load(seg_path)
    
    vol_tensor = torch.FloatTensor(vol_arr)
    seg_tensor = torch.FloatTensor(seg_arr)
    return seg_tensor
    

In [ ]:
seg_tensor = list(map(get_item, tqdm(range(df.shape[0]))))


100%|██████████| 210/210 [02:31<00:00,  1.39it/s]


In [ ]:
print(seg_tensor[0].shape)

torch.Size([87, 256, 256])


In [ ]:
organ_df = list(map(check_nonzero_slice, seg_tensor))
stat_df = pd.DataFrame(organ_df)
stat_df.describe()


,nonzero_z_min,nonzero_z_max,nonzero_y_min,nonzero_y_max,nonzero_x_min,nonzero_x_max
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,35.171429,78.033333,101.500000,165.380952,55.990476,203.757143
std,24.048777,25.865938,15.720115,10.563439,12.003983,15.483069
min,3.000000,40.000000,50.000000,139.000000,21.000000,95.000000
25%,23.000000,62.250000,92.000000,158.000000,48.250000,197.000000
50%,31.000000,74.000000,102.000000,166.000000,56.000000,205.000000
75%,38.000000,83.000000,113.000000,171.000000,62.750000,212.000000
max,121.000000,171.000000,140.000000,193.000000,126.000000,245.000000


In [ ]:
stat_df.to_csv("/content/drive/MyDrive/Seg3D/KiTS2019/kits19/output/stat_preprprocess.csv")

In [ ]:
z_min = [(35.171429 - 24.048777), (35.171429 + 24.048777)] 
z_max = [(78.033333 - 25.865938), (78.033333 + 25.865938)]
print(np.round(z_min))
print(np.round(z_max))

[11. 59.]
[ 52. 104.]


In [ ]:
y_min = [(103.180952 - 15.597375), (103.180952 + 15.597375)] 
y_max = [(165.423810 - 13.062837), (165.423810 + 13.062837)]
print(np.round(y_min))
print(np.round(y_max))

In [ ]:
x_min = [(55.642857 - 13.806630), (55.642857 + 13.806630)] 
x_max = [(200.104762 - 16.316794), (200.104762 + 16.316794)]
print(np.round(x_min))
print(np.round(x_max))

In [ ]:
import numpy as np

a = np.random.rand(2,3,2)

In [ ]:
a

array([[[0.21488007, 0.21768973],
        [0.81116842, 0.86517395],
        [0.06874115, 0.87335622]],

       [[0.25269353, 0.19014854],
        [0.52220371, 0.56854066],
        [0.99844149, 0.2956776 ]]])

In [ ]:
max_pos = np.argmax(a, axis=0)
max_pos

array([[1, 0],
       [0, 0],
       [1, 0]])

In [ ]:
max_pos.shape

(3, 2)

In [ ]:
idc = 2
pos = np.where(max_pos==max_pos[idc])[0]
new_arr = np.zeros_like(a[idc])  # Create a new array of zeros with the same shape as arr
new_arr[pos] = 1  # Set the maximum position to 1
new_arr

IndexError: ignored

In [ ]:
import numpy as np

arr = np.array([[5, 2, 7], [1, 9, 3],
                [8, 4, 6], [0, 2, 1]])

print(arr.shape)
print(arr)
max_pos = np.unravel_index(arr.argmax(axis=), arr.shape, order="F")  # Find the position of the maximum element

new_arr = np.zeros_like(arr)  # Create a new array of zeros with the same shape as arr
new_arr[max_pos] = 1  # Set the maximum position to 1

print(new_arr)
print(new_arr.shape)


(4, 3)
[[5 2 7]
 [1 9 3]
 [8 4 6]
 [0 2 1]]
[[1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 0]]
(4, 3)


In [ ]:
torch.max(a, dim=0)

torch.return_types.max(
values=tensor([0.6467, 0.9873, 0.6824, 0.8887, 0.6719, 0.7559]),
indices=tensor([0, 1, 0, 2, 1, 0]))

In [ ]:
1, 0, 1, 0, 0, 1
0, 1, 0, 0, 1, 0
0, 0, 0, 1, 0, 0

In [ ]:
a

tensor([[0.6467, 0.6362, 0.6824, 0.0202, 0.3696, 0.7559],
        [0.1481, 0.9873, 0.2172, 0.7545, 0.6719, 0.6821],
        [0.5620, 0.2530, 0.3296, 0.8887, 0.4892, 0.2633]])